Лабораторная работа. MySQL.

Эта лабораторная работа имеет целью эмулировать процесс обработки данных, загрузку их в базу данных, обновление этих данных в базе, составление запросов к ней и выгрузку данных. Создавая и заполняя таблицы, проверяйте появление тех или иных элементов или записей в таблице в MySQL Workbench для большей наглядности.

Смодулируем небольшое исследование, которое разметит тексты по нескольким метрикам. Для этого используем небольшой датасет критики детской литературы отсюда: https://dataverse.pushdom.ru/dataset.xhtml?persistentId=doi:10.31860/openlit-2022.12-B008

**Глава 1. Метаданные**

Пушкинский дом подготавливает для нас описание корпуса, поэтому можно использовать табличку с метаданными для нашей работы.

In [ ]:
import pandas as pd

In [ ]:
path = 'bibliography.tsv' #измените путь на свой
meta = pd.read_csv(path, sep='\t') #пушкинский дом выкладывает файлы в  tsv (tab separated values), поэтому надо указать правильный делиметер
meta

В таблице есть записи годов "1927-1928". Это не годится для типа данных INT или YEAR в SQL таблице, поэтому изменим слегка эту запись:

In [ ]:
meta['year'] = meta['year'].replace(['1927-1928'], '1928')

Как видно, в табличке есть вся нужная метаинформация по корпусу. Создадим отдельную базу данных и сохраним туда эту табличку. Назначим номер первичным ключом.

In [ ]:
%pip install mysql-connector-python

In [ ]:
import mysql.connector
from getpass import getpass

In [ ]:
mydb = mysql.connector.connect(
    host = 'localhost',
    user = "root",
    password = getpass('Введите ваш пароль!')  
)

mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES")

for database in mycursor:
  print(database)

In [ ]:
mydb = mysql.connector.connect(
  host = "localhost",
  user = "root",
  password = getpass('Введите ваш пароль!')   
)

mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE critics") #создаём пока только саму БД с названием critics

Загрузим табличку с метаинформацией в БД. Заполним её таким же образом, как было в семинарском занятии по Python + SQL. Библиографию брать не будем. Для этого извлечем кортежи, т.е. строки, из пандасовского датафрейма.

In [ ]:
for_db = [(int(meta['№'][ind]), meta['author'][ind], meta['title'][ind], meta['source'][ind], int(meta['year'][ind])) for ind in meta.index]
for_db #list comprehension для сбора кортежей для нашей таблицы

Создадим пустую табличку.

Для начала посмотрим, какие таблички уже есть в БД:

In [ ]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")

for table in mycursor:
  print(table)

Пусто! Создадим таблицу.

In [ ]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE meta (id INT AUTO_INCREMENT PRIMARY KEY,  author VARCHAR(1000), title VARCHAR(1000), source VARCHAR(1000), year YEAR)")

Проверим еще раз.

In [ ]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")

for table in mycursor:
  print(table)

Теперь заполним эту таблицу полученной ранее информацией.

In [ ]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
sql = "INSERT INTO meta (id, author, title, source, year) VALUES (%s, %s, %s, %s, %s)"

mycursor.executemany(sql, for_db) 
mydb.commit() #не забудьте делать коммит, иначе не сохранится запись в БД

Посмотрим, что получилось. Пока только в Workbench.

**Глава 2. Хранение текстов в БД**

Создадим ещё одну таблицу в нашей БД, в неё мы запишем тексты критических разборов. Подготовим некоторые данные для этого.

In [ ]:
import os

Создадим список из кортежей, состоящих из индекса и текста рецензии.

In [ ]:
path = 'critic_corpus\\' #не забудьте поменять на путь, где лежат ваши тексты
texts = [] #сюда мы сохраним кортежи текстов и их индексов
counter = 1 #счётчик, который поможет нам сохранять индексы
for filename in os.listdir(path): #проходим циклом по названиям файлов в папке
    with open(path + filename, encoding='utf-8') as txt:
        texts.append((counter, txt.read()))
        counter += 1
        
texts

Создадим табличку texts и заполним её текстами.

In [ ]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE texts (id INT AUTO_INCREMENT PRIMARY KEY,  text TEXT)")
sql = "INSERT INTO texts (id, text) VALUES (%s, %s)"
mycursor.executemany(sql, texts) 
mydb.commit()

Не забудем связать таблицы через внешний ключ:

In [ ]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
mycursor.execute("ALTER TABLE texts ADD FOREIGN KEY (id) REFERENCES meta(id)")

Теперь мы можем забыть о том, что у нас есть какая-то папочка и работать только с сервером. Перейдем к следующему этапу.

**Глава 3. Выгрузка данных из БД и их обработка**

Сделаем небольшие подсчёты по текстам нашей БД. Выгрузим тексты из неё.

In [ ]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
sql = "SELECT text FROM texts"
mycursor.execute(sql)
myresult = mycursor.fetchall()

Тексты в myresult лежат в кортежах, достанем их из них:

In [ ]:
texts_cr = []
for tupl in myresult:
   # print(tupl)
    for txt in tupl: 
        #print(txt)
        texts_cr.append(txt)
texts_cr

Подсчитаем пару простых метрик. Например, ридабилити. Не будем вдаваться в сложности и тонкости правильных формул, воспользуемся готовыми решениями для скорости. Посчитаем метрику MTLD (одна из метрик лексического разнообразия) TTR (type-token ratio).

In [ ]:
%pip install setuptools

In [ ]:
%pip install lexical_diversity

In [ ]:
from lexical_diversity import lex_div as ld

In [ ]:
lst_mtlds_ttrs = []
counter = 1
for text in texts_cr:
    tokenized_text = ld.tokenize(text)
    tokenized_text_mtld = round(ld.mtld(tokenized_text), 3)
    tokenized_text_ttr = round(ld.mattr(tokenized_text), 3)
    lst_mtlds_ttrs.append((counter, tokenized_text_mtld, tokenized_text_ttr))
    counter += 1
lst_mtlds_ttrs #эти значения мы запишем затем в нашу БД

Создадим ещё одну табличку, где будем хранить наши данные для аналитики.

In [ ]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE analytics (id INT AUTO_INCREMENT PRIMARY KEY,  mtld FLOAT(3), ttr FLOAT(3))")
#mycursor.execute("ALTER TABLE analytics ADD FOREIGN KEY (id) REFERENCES meta(id)")
sql = "INSERT INTO analytics (id, mtld, ttr) VALUES (%s, %s, %s)"
mycursor.executemany(sql, lst_mtlds_ttrs) 
mydb.commit()

In [ ]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
mycursor.execute("ALTER TABLE analytics ADD FOREIGN KEY (id) REFERENCES meta(id)")

**Глава 4. Аналитика.**

Все наши дальнейшие действия крайне тривиальны: нам нужно получать выгрузку и что-то анализировать. Давайте посчитаем корреляции по записям в нашей БД.

In [ ]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
sql = "SELECT mtld, ttr FROM analytics"
mycursor.execute(sql)
myresult = mycursor.fetchall()

In [ ]:
import numpy as np

In [ ]:
mtlds = [] #сохраним отдельно mtld и ttr
ttrs = []
for tupl in myresult:
    mtlds.append(tupl[0])
    ttrs.append(tupl[1])

In [ ]:
cor_matrix = np.corrcoef(mtlds, ttrs) #делаем матрицу корреляций
cor_matrix #получаем среднюю (среднеслабую) корреляцию в 0.46

Так как мы в SQL, мы можем писать более хитрые запросы. Например, можно вывести значения mtld и ttr текстов, написанных в и после 1927 года. Это так называемые вложенные запросы, или подзапросы.

In [ ]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
sql = "SELECT mtld, ttr FROM analytics WHERE id IN (SELECT id FROM meta WHERE year >= 1927)"
mycursor.execute(sql)
myresult = mycursor.fetchall()

In [ ]:
print(len(myresult))

Как видно после исполнения предыдущей команды, в результат выдачи попало только 126 записей. Таким образом мы отсекли около половины текстов. Давайте снова посчитаем корреляцию:

In [ ]:
mtlds = [] #сохраним отдельно mtld и ttr
ttrs = []
for tupl in myresult:
    mtlds.append(tupl[0])
    ttrs.append(tupl[1])
r = np.corrcoef(mtlds, ttrs) #делаем матрицу корреляций
r #получаем среднюю (среднеслабую) корреляцию в 0.46

Довольно забавно, но степень корреляции упала. У этого нет какого-то научного или серьёзного значения, т.к. это совсем небольшое и игрушечное исследование, но тем не менее примерно так вы и будете работать в тех научных проектах, где есть БД, а также примерно такие умения начального и среднего уровнвя от вас обычно ожидается на стартовых позициях, где требуется знание SQL.

**Глава 5. Свободное искусство (то есть задание для самостоятельной работы).**

Ваша задача выбрать какой-либо датасет (не слишком большой - вы не должны сделать итоговый проект в рамках этой работы, хотя итоговый проект во многом должен быть похож именно на эту лабораторную работу) из множества датасетов Пушкинского дома ( https://dataverse.pushdom.ru/) и обработать его.

Задания:
1) выбрать датасет;
2) создать БД с несколькими таблицами, которые будут отображать всё великолепие и многообразие датасета (мета-, тексты, статистические подсчёты);
3) заполнить эти таблицы;
4) придумать небольшое исследование: например, корреляцию количества определенных частей речи с количеством повторений слова "запонка" в тексте. Достаточно 1-2 метрик, это игрушечное исследование, а не заготовка вашей статьи;
5) проиллюстрировать ваше умение создавать таблицы, заполнять таблицы, получать выгрузку, обновлять таблицы и данные в них;
6) также напишите 5-6 комбинаторных запросов. Например, "подсчёт числа текстов, написанных до 1927 года". Попробуйте придумать максимально сложные для рассматриваемого вами датасета;
7) Напишите хотя бы один вложенный запрос;
8) Сделайте один разумный JOIN и обоснуйте его.

Оцениваться будет ваше умение аккуратно писать и обосновывать запросы. Комментируйте большинство строчек кода, надписав, что делает эта строчка. Каждый недублирующийся SQL запрос распишите, что он последовательно делает.

Помимо семинарских занятий 3 модуля вам могут быть полезны выложенные учебники, а также онлайн-справочники, а особенно следующие  главы:
1) https://www.w3schools.com/sql/default.asp
2) https://www.w3schools.com/sql/sql_join.asp
3) https://www.w3schools.com/sql/sql_alter.asp
4) https://www.w3schools.com/sql/sql_insert.asp
5) https://www.w3schools.com/sql/sql_update.asp


Пришлите до 15.05 включительно на почту a.klimov@hse.ru архив с такой же тетрадкой, где выполнены и прокомментированы задания, а также дамп и схема вашей БД. По желанию можно прислать csv таблицы выгрузок.

In [ ]:
#ваш код здесь